In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib as mpl
from scipy.stats import norm

# 中文乱码的处理
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']# 设置微软雅黑字体
plt.rcParams['axes.unicode_minus'] = False # 避免坐标轴不能正常的显示负号

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
# from xgboost import XGBRegressor

# Road the data
X_full = pd.read_csv('./data/train.csv')
X_test_full = pd.read_csv('./data/test.csv')
sub = pd.read_csv('./data/sub.csv')
print(X_full.shape, X_test_full.shape)
#$$


(59288, 26) (14823, 25)


In [3]:
# X_full.head();
X_full.columns

Index(['数据ID', '容纳人数', '便利设施', '洗手间数量', '床的数量', '床的类型', '卧室数量', '取消条款', '所在城市',
       '清洁费', '首次评论日期', '房主是否有个人资料图片', '房主身份是否验证', '房主回复率', '何时成为房主',
       '是否支持随即预订', '最近评论日期', '维度', '经度', '民宿周边', '评论个数', '房产类型', '民宿评分', '房型',
       '邮编', '价格'],
      dtype='object')

In [4]:
import re
words = [re.findall("\w+",  str) for str in X_full.便利设施]

In [5]:
#

add new col, count for bianlisheshi


In [6]:
X_full['new'] = 1
for i in range(0, len(X_full)):
    X_full.loc[i,'new'] = len((X_full.便利设施[i]).split())

In [7]:
X_full['new_len'] = 1
for i in range(0, len(X_full)):
    X_full.loc[i,'new_len'] = len((X_full.便利设施[i]))

In [8]:
#

check na

In [9]:
# na_index = [X_full[col].isnull().values.sum() for col in X_full.columns]
# na_key = X_full.columns
# na_frame = pd.DataFrame(na_index, na_key)

Cat and Num split


In [10]:
X_full = X_full.drop('数据ID', axis = 1)

In [11]:
categorical_cols = [cname for cname in X_full.columns if 
X_full[cname].dtype == 'object']

# Select numerical columns
numerical_cols = [cname for cname in X_full.columns if
                  X_full[cname].dtype in ['float64', 'int64']]

In [12]:
categorical_cols

['便利设施',
 '首次评论日期',
 '房主是否有个人资料图片',
 '房主身份是否验证',
 '房主回复率',
 '何时成为房主',
 '最近评论日期',
 '民宿周边',
 '邮编']

In [42]:
X_full['首次评论日期'] = X_full['首次评论日期'].astype(str)
X_full['何时成为房主'] = X_full['何时成为房主'].astype(str)
X_full['最近评论日期'] = X_full['最近评论日期'].astype(str)

In [22]:
# 
X_full['首次评论日期'] = [re.findall("^\d{4}\-\d{2}", str) for str in X_full.首次评论日期]
X_full['何时成为房主'] = [re.findall("^\d{4}\-\d{2}", str) for str in X_full.何时成为房主]
X_full['最近评论日期'] = [re.findall("^\d{4}\-\d{2}", str) for str in X_full.最近评论日期]

In [56]:
X_full['首次评论日期_new'] = [i.strip("['']") for i in X_full.首次评论日期.to_list()]
X_full['何时成为房主_new'] = [i.strip("['']") for i in X_full.何时成为房主.to_list()]
X_full['最近评论日期_new'] = [i.strip("['']") for i in X_full.最近评论日期.to_list()]

In [95]:
# [isna(i) for i in X_full.房主回复率]
X_full['房主回复率_new'] = X_full.房主回复率.isnull()

In [96]:
X_full['房主回复率_new']

0         True
1         True
2        False
3         True
4        False
         ...  
59283    False
59284    False
59285    False
59286    False
59287    False
Name: 房主回复率_new, Length: 59288, dtype: bool

 ---

In [13]:
# X_full.首次评论日期 = [str(i) for i in X_full.首次评论日期]
# X_full.首次评论日期 = (re.findall("^\d{4}\-\d{2}", str) for str in X_full.首次评论日期)


In [36]:
# test_df['Season'] = test_df['ID'].map(lambda x: int(x[:4]))

In [38]:
# X_full['bnenwne'] = X_full['new'].map(lambda x: str(x)[:1])

#--------------------------------------------
drop

In [ ]:
X_full.drop(['便利设施', '最近评论日期', '何时成为房主', '首次评论日期', '房主回复率_new'])

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
y = X_full['价格']
X_full.pop('价格')

In [ ]:
numerical_transformer = SimpleImputer(strategy='most_frequent')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
X_train_new = preprocessor.fit_transform(X_full)

In [ ]:
X_train_new

In [ ]:
# X_full.columns

cat one hot

In [ ]:
# from sklearn.model_selection import train_test_split

In [ ]:
# X_train_full, X_valid_full, y_train, y_valid = 
# train_test_split(X_full, y, train_size=0.8, test_size=0.2, random_state=0)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint as sp_randint


In [ ]:
my_reg = RandomForestRegressor()
scores = -1 * cross_val_score(my_reg, 
    X_train_new, y,
    cv=5, n_jobs = -1,
    scoring='neg_root_mean_squared_error')

print("MAE scores:\n", scores)
print("Average MAE score (across experiments):")
print(scores.mean())

In [ ]:
hp = {"max_depth": [14, 11, 9],
              "min_samples_split": sp_randint(2, 11)
            #   "criterion": ["gini", "entropy"],
            #   "bootstrap": [True, False]
              }

# run randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(my_reg, param_distributions=hp,
                                   n_iter=n_iter_search, cv=5)



In [ ]:
 
#% Initial Models
# from sklearn.ensemble import RandomForestRegressor
# import lightgbm as lgb

RFReg = RandomForestRegressor(n_estimators = 10, criterion = "mse", max_depth = 10, n_jobs = -1,random_state = 0).fit(X_train_new, y)



test data

In [ ]:
X_test_full['new'] = 1
for i in range(0, len(X_test_full)):
    X_test_full.loc[i,'new'] = len((X_test_full.便利设施[i]).split())

In [ ]:
X_test_full['new_len'] = 1
for i in range(0, len(X_test_full)):
    X_test_full.loc[i,'new_len'] = len((X_test_full.便利设施[i]))

In [ ]:
X_test_full = X_test_full.drop('数据ID', axis = 1)

In [ ]:
categorical_cols = [cname for cname in X_test_full.columns if 
X_test_full[cname].dtype == 'object']

# Select numerical columns
numerical_cols = [cname for cname in X_test_full.columns if
                  X_test_full[cname].dtype in ['float64', 'int64']]

In [ ]:
# y = X_test_full['价格']
# X_test_full.pop('价格')

In [ ]:
X_test_new = preprocessor.transform(X_test_full)

In [ ]:
predicted_ = RFReg.predict(X_test_new)

In [ ]:
X_test_new